In [ ]:
import xarray as xr
from matplotlib import pyplot as plt
import cftime._cftime as cft

| Experiment Name | Description | Parameters |
| :- | :- | :- |
| EXP_z   |   No changes in usrdef                                |                                                           |
| EXP_z2  |   set cutoff to 1000m                                 |                                                           |

In [ ]:
path = "/home/dkamm/NEMOGCM/tests/BASIN2/"

In [ ]:
def clean_coords(xrdataset, start_year):
    try:
        shift = cft.timedelta(days=360 * start_year)
        datetimeindex = (xrdataset.indexes['time_counter'] + shift).to_datetimeindex
    except:
        datetimeindex = xrdataset.indexes['time_counter']
    try:
        latitude  = xrdataset.nav_lat[:,0].values
        longitude = xrdataset.nav_lon[0,:].values
    except:
        latitude  = xrdataset.nav_lat_grid_T[:,0].values
        longitude = xrdataset.nav_lon_grid_T[0,:].values
    try:
        xrdataset = xrdataset.rename_dims({'x_grid_T': 'x' ,'y_grid_T': 'y'})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3t.cumsum('deptht')})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3u.cumsum('depthu')})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3v.cumsum('depthv')})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3w.cumsum('depthw')})
    except:
        pass
    return xrdataset.assign_coords(dict({'time_counter':datetimeindex, 'y': latitude, 'x': longitude}))

In [ ]:
T = clean_coords(xr.load_dataset(path + 'EXP_z/BASIN_grid_T.nc'), 2000)
T2 = clean_coords(xr.load_dataset(path + 'EXP_z2/BASIN_grid_T.nc'), 2000)

In [ ]:
domain = clean_coords(xr.load_dataset(path + 'EXP_z/domain_cfg_out.nc'), 2000)
domain2 = clean_coords(xr.load_dataset(path + 'EXP_z2/domain_cfg_out.nc'), 2000)

In [ ]:
import numpy as np
plt.figure()
plt.plot(np.arange(T2.deptht.count()),T2.deptht.values, label='z-coords to 50m', color='darkred')
plt.plot(np.arange(T.deptht.count()),T.deptht.values, label='z-coords to 1000m', color='midnightblue')
plt.title('1D depth coordinate')
plt.legend()
plt.grid()

In [ ]:
import numpy as np
plt.figure()
plt.plot(np.arange(domain2.e3w_0.isel(x=2, y=2, time_counter=0).count()),domain2.e3w_0.isel(x=2, y=2, time_counter=0).cumsum().values, label='Shallow point', color='darkred')
plt.plot(np.arange(domain2.e3w_0.isel(x=20, y=20, time_counter=0).count()),domain2.e3w_0.isel(x=20, y=20, time_counter=0).cumsum().values, label='Deep point', color='midnightblue')
plt.legend()
plt.grid()

In [ ]:
domain2.e3w_0.sum('nav_lev').plot()